In [1]:
import pandas as pd
import gzip
import time
# Install a few python packages using pip
from common import utils
utils.require_package('nltk')
utils.require_package("wget")      # for fetching dataset

In [2]:
# Standard python helper libraries.
from __future__ import print_function
from __future__ import division
import os, sys, time
import collections
import itertools

# Numerical manipulation libraries.
import numpy as np
from scipy import stats, optimize

# NLTK is the Natural Language Toolkit, and contains several language datasets
# as well as implementations of many popular NLP algorithms.
# HINT: You should look at what is available here when thinking about your project!
import nltk

# Helper libraries (see the corresponding py files in this notebook's directory).
from common import utils, vocabulary
import segment

utils.require_package("tqdm")  # for nice progress bars
from tqdm import tqdm as ProgressBar

In [3]:


def parse(path):
  print('start parse')
  start_parse = time.time()
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)
  end_parse = time.time()
  print('end parse with time for parse',end_parse - start_parse)

def getDF(path):
  print('start getDF')
  start = time.time()
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  print('end getDF')
  end = time.time()
  print('time taken to load data = ',end-start)
  return pd.DataFrame.from_dict(df, orient='index')
df = getDF('reviews_Toys_and_Games.json.gz')

start getDF
start parse
end parse with time for parse 129.83230781555176
end getDF
time taken to load data =  129.83265256881714


In [4]:
print(df.shape)
print(df.columns)
df.head(10)

(2252771, 9)
Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AMEVO2LY6VEJA,0000191639,Nicole Soeder,"[0, 0]","Great product, thank you! Our son loved the pu...",5.0,Puzzles,1388016000,"12 26, 2013"
1,A3C9CSW3TJITGT,0005069491,Renee,"[0, 0]",I love these felt nursery rhyme characters and...,4.0,Charming characters but busy work required,1377561600,"08 27, 2013"
2,A31POTIYCKSZ9G,0076561046,So CA Teacher,"[0, 0]",I see no directions for its use. Therefore I h...,3.0,No directions for use...,1404864000,"07 9, 2014"
3,A2GGHHME9B6W4O,0131358936,Dalilah G.,"[0, 0]",This is a great tool for any teacher using the...,5.0,Great CD-ROM,1382400000,"10 22, 2013"
4,A1FSLDH43ORWZP,0133642984,Dayna English,"[0, 0]",Although not as streamlined as the Algebra I m...,5.0,Algebra II -- presentation materials,1374278400,"07 20, 2013"
5,A3NXHTSEBX9YHB,0279515766,marlenetbueras,"[0, 0]",i am glad to get it after 25 yr of waiting for...,5.0,my strawberry shortcake,1363564800,"03 18, 2013"
6,AYVR1MQCTNU5D,0375829695,annie,"[0, 0]",What a great theme for a puzzle book. My daugh...,5.0,So cute!!,1291939200,"12 10, 2010"
7,A3CJHKFHHQJP2K,0375829695,"Beth Sharo ""bookmom""","[0, 0]",My son got this book for his birthday. He lov...,1.0,Disappointing Puzzle Book,1297209600,"02 9, 2011"
8,A3638FINP26E8N,0375829695,C. Boykin,"[1, 1]",Love the book format. My 4yr old grandson love...,3.0,frustrating puzzle pieces,1282521600,"08 23, 2010"
9,AUT7723O49VMN,0375829695,Cindy Lindy,"[1, 1]",I'm a 2nd grade teacher. My students found th...,4.0,challenging and fun,1237766400,"03 23, 2009"


In [6]:
print(df.groupby('overall').count())

         reviewerID     asin  reviewerName  helpful  reviewText  summary  \
overall                                                                    
1.0          192993   192993        192435   192993      192993   192993   
2.0          115801   115801        115416   115801      115801   115801   
3.0          193941   193941        193195   193941      193941   193941   
4.0          407884   407884        406255   407884      407884   407884   
5.0         1342152  1342152       1333623  1342152     1342152  1342152   

         unixReviewTime  reviewTime  
overall                              
1.0              192993      192993  
2.0              115801      115801  
3.0              193941      193941  
4.0              407884      407884  
5.0             1342152     1342152  


In [7]:
for i in range(10):
    print(df['reviewerID'].iloc[i])
    print(df['reviewText'].iloc[i])

AMEVO2LY6VEJA
Great product, thank you! Our son loved the puzzles.  They have large pieces yet they are still challenging for a 4 year old.
A3C9CSW3TJITGT
I love these felt nursery rhyme characters and scenes.  The quality of the felt is good, and the illustrations are detailed and pretty.  As noted, the figures and scenes are printed on 2 large sheets of flannel and each individual item needs to be cut out.  This process took me 2 hours of tiny cutting.  To me it does not lend itself to a book form but rather laying out the scenes separately or for use on a flannel board.  However, I love the quiet play it offers for my toddler, and as a former Kindergarten teacher, I understand the value of learning rhyme and its connection to future reading.  Overall, delightful product with some work involved.
A31POTIYCKSZ9G
I see no directions for its use. Therefore I have to make up the games, unfortunately.
A2GGHHME9B6W4O
This is a great tool for any teacher using the Pre Calculus Enhanced with 

In [8]:
from sklearn.model_selection import train_test_split
train_toys,devtest = train_test_split(df, test_size=0.4)
dev_toys,test_toys = train_test_split(devtest,test_size = 0.5)
print(train_toys.shape,dev_toys.shape,test_toys.shape)

(1351662, 9) (450554, 9) (450555, 9)


In [9]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /home/arunima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
size_train = 1200000
len_max_train = train_toys[train_toys.overall!=3].shape[0] #max possible length of train data set taking out the 3 ratings.
print("Number of reviews with ratings != 3 in train set",len_max_train)
temp_size_train = min(len_max_train,size_train)

len_max_dev = dev_toys[dev_toys.overall!=3].shape[0]
print("Number of reviews with ratings != 3 in dev set",len_max_dev)
temp_size_dev = min(len_max_dev,int(0.3*temp_size_train)) #making the dev set about 0.3 times the train set.

temp_train_toys = train_toys[train_toys.overall != 3][:temp_size_train]
print(temp_train_toys.shape)
print(temp_train_toys.groupby('overall').count())
#print(temp_train_toys[:5])

temp_dev_toys = dev_toys[dev_toys.overall!=3][:temp_size_dev]
print(temp_dev_toys.shape)
print(temp_dev_toys.groupby('overall').count())
print(temp_dev_toys[:3])

Number of reviews with ratings != 3 in train set 1235735
Number of reviews with ratings != 3 in dev set 411501
(1200000, 9)
         reviewerID    asin  reviewerName  helpful  reviewText  summary  \
overall                                                                   
1.0          112494  112494        112179   112494      112494   112494   
2.0           67561   67561         67331    67561       67561    67561   
4.0          237627  237627        236669   237627      237627   237627   
5.0          782318  782318        777382   782318      782318   782318   

         unixReviewTime  reviewTime  
overall                              
1.0              112494      112494  
2.0               67561       67561  
4.0              237627      237627  
5.0              782318      782318  
(360000, 9)
         reviewerID    asin  reviewerName  helpful  reviewText  summary  \
overall                                                                   
1.0           33967   33967        

In [29]:
cnt = collections.Counter()
#print(ds.vocab.num_unigrams)
x_tokens_list = []
start = time.time()
for i in range(temp_train_toys.shape[0]):
#for i in range(5):
    x_tokens = word_tokenize(temp_train_toys.reviewText.iloc[i])
    #print(train_toys.reviewText.iloc[i],x_tokens)
    #print(type(x_tokens),len(x_tokens))
    x_tokens_canonical = utils.canonicalize_words(x_tokens)
    #print(x_tokens_canonical,type(x_tokens_canonical),len(x_tokens_canonical))
    x_tokens_list.append(x_tokens_canonical)
    #print('list',x_tokens_list)
    #print(x_tokens_list.shape)
    for word in x_tokens_canonical:
        cnt[word]+=1
    if i%25000 == 0:
        #print('x_tokens',x_tokens,'x_tokens_canonical',x_tokens_canonical)
        print('done',i)
end = time.time()
print('total time taken to canonicalize',start-end)

print('100 most common words in the dataset:',cnt.most_common(100))

done 0
done 10000
done 20000
done 30000
done 40000
done 50000
done 60000
done 70000
done 80000
done 90000
done 100000
done 110000
done 120000
done 130000
done 140000
done 150000
done 160000
done 170000
done 180000
done 190000
done 200000
done 210000
done 220000
done 230000
done 240000
done 250000
done 260000
done 270000
done 280000
done 290000
done 300000
done 310000
done 320000
done 330000
done 340000
done 350000
done 360000
done 370000
done 380000
done 390000
done 400000
done 410000
done 420000
done 430000
done 440000
done 450000
done 460000
done 470000
done 480000
done 490000
done 500000
done 510000
done 520000
done 530000
done 540000
done 550000
done 560000
done 570000
done 580000
done 590000
done 600000
done 610000
done 620000
done 630000
done 640000
done 650000
done 660000
done 670000
done 680000
done 690000
done 700000
done 710000
done 720000
done 730000
done 740000
done 750000
done 760000
done 770000
done 780000
done 790000
done 800000
done 810000
done 820000
done 830000
done 8

In [30]:
print(len(x_tokens_list))
print(x_tokens_list[:2])
print(x_tokens_canonical[:2])

1200000
[['would', 'have', 'had', 'more', 'fun', 'with', 'him', 'if', 'more', 'people', 'knew', 'who', 'he', 'was', '.', 'they', 'will', 'learn', ',', 'when', 'the', 'stars', 'are', 'right', '!'], ['i', 'had', 'this', 'figure', 'as', 'a', 'kid', 'and', 'just', 'recently', 'i', 'decided', 'to', 'relive', 'some', 'of', 'my', 'childhood', 'and', 'found', 'one', 'on', 'here', 'for', 'a', 'good', 'price', '.', 'figure', 'is', 'just', 'like', 'i', 'remember', ',', 'simple', 'but', 'yet', 'really', 'awesome', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!']]
['i', 'am']


In [31]:
cnt_dev = collections.Counter()
#print(ds.vocab.num_unigrams)
x_tokens_list_dev = []
start = time.time()
for i in range(temp_dev_toys.shape[0]):
#for i in range(5):
    x_tokens = word_tokenize(temp_dev_toys.reviewText.iloc[i])
    #print(train_toys.reviewText.iloc[i],x_tokens)
    #print(type(x_tokens),len(x_tokens))
    x_tokens_canonical = utils.canonicalize_words(x_tokens)
    #print(x_tokens_canonical,type(x_tokens_canonical),len(x_tokens_canonical))
    x_tokens_list_dev.append(x_tokens_canonical)
    #print('list',x_tokens_list)
    #print(x_tokens_list.shape)
    for word in x_tokens_canonical:
        cnt[word]+=1
    if i%25000 == 0:
        #print('x_tokens',x_tokens,'x_tokens_canonical',x_tokens_canonical)
        print('done',i)
end = time.time()
print('total time taken to canonicalize dev set',start-end)

print('100 most common words in the dataset:',cnt.most_common(100))

done 0
done 10000
done 20000
done 30000
done 40000
done 50000
done 60000
done 70000
done 80000
done 90000
done 100000
done 110000
done 120000
done 130000
done 140000
done 150000
done 160000
done 170000
done 180000
done 190000
done 200000
done 210000
done 220000
done 230000
done 240000
done 250000
done 260000
done 270000
done 280000
done 290000
done 300000
done 310000
done 320000
done 330000
done 340000
done 350000
total time taken to canonicalize dev set -339.69965076446533
100 most common words in the dataset: [('.', 5515451), ('the', 4837349), (',', 3103855), ('and', 3099236), ('it', 3019358), ('a', 2575808), ('to', 2552900), ('i', 2402632), ('is', 1757049), ('this', 1715868), ('for', 1683699), ('of', 1504924), ('my', 1231933), ('!', 1130799), ('with', 1077799), ('in', 978953), ('that', 935978), ('was', 885707), ('you', 737209), ('but', 721487), ('on', 713808), ('are', 683218), ('have', 651538), ("'s", 636324), ('DG', 625355), ('not', 619587), ('as', 611063), ('so', 583818), ('they',

In [32]:
#Create vocabulary on train data set.
vocab = vocabulary.Vocabulary(cnt, size=None)
print('vocab size',vocab.size)
print('100 most common words in the dataset:',cnt.most_common(100))

x_tokens_id_list = []
for i in range(temp_train_toys.shape[0]):
#for i in range(5):
    x_tokens_ids = vocab.words_to_ids(x_tokens_list[i])
    #print (x_tokens_ids)
    x_tokens_id_list.append(x_tokens_ids)
    #print(x_tokens_id_list)

print('id list shape',len(x_tokens_id_list))

print('doing dev set')
x_tokens_id_list_dev = []
for i in range(temp_dev_toys.shape[0]):
#for i in range(5):
    x_tokens_ids = vocab.words_to_ids(x_tokens_list_dev[i])
    #print (x_tokens_ids)
    x_tokens_id_list_dev.append(x_tokens_ids)
    #print(x_tokens_id_list_dev)
print('dev list shape',len(x_tokens_id_list_dev))


vocab size 674074
100 most common words in the dataset: [('.', 5515451), ('the', 4837349), (',', 3103855), ('and', 3099236), ('it', 3019358), ('a', 2575808), ('to', 2552900), ('i', 2402632), ('is', 1757049), ('this', 1715868), ('for', 1683699), ('of', 1504924), ('my', 1231933), ('!', 1130799), ('with', 1077799), ('in', 978953), ('that', 935978), ('was', 885707), ('you', 737209), ('but', 721487), ('on', 713808), ('are', 683218), ('have', 651538), ("'s", 636324), ('DG', 625355), ('not', 619587), ('as', 611063), ('so', 583818), ('they', 575303), ("n't", 571841), ('we', 504222), ('he', 491747), ('very', 483348), ('one', 471937), ('great', 454262), ('be', 449318), ('all', 408473), (')', 405679), ('she', 401490), ('DGDG', 391092), ('them', 381278), ('(', 361572), ('would', 357256), ('just', 348920), ('can', 348020), ('old', 347236), ('at', 336834), ('like', 336179), ('has', 331586), ('game', 330565), ('toy', 324293), ('do', 319154), ('when', 313682), ('her', 305692), ('up', 305368), ('or', 2

In [33]:
#Converting ratings to binary
temp_train_y = np.zeros(temp_train_toys.shape[0])
temp_train_y[temp_train_toys.overall > 3] = 1
#print(temp_train_y.shape,temp_train_y[:20],temp_train_toys.overall[:20])
temp_dev_y = np.zeros(temp_dev_toys.shape[0])
temp_dev_y[temp_dev_toys.overall>3] = 1
print(temp_dev_y.shape)
print(temp_dev_y[:20],temp_dev_toys.overall[:20])

(360000,)
[ 1.  1.  0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.
  1.  1.] 392707     5.0
1253479    5.0
304445     1.0
865041     4.0
1457898    5.0
799874     5.0
1977200    1.0
2243413    5.0
2094383    5.0
341215     5.0
1337643    4.0
222962     5.0
798888     5.0
2088004    5.0
1251549    5.0
1420047    4.0
2055081    4.0
2175931    2.0
570459     4.0
291418     5.0
Name: overall, dtype: float64


In [34]:
# x = utils.id_lists_to_sparse_bow(df['ids'], self.vocab.size)
# y = np.array(df.label, dtype=np.int32)
    
train_x_csr = utils.id_lists_to_sparse_bow(x_tokens_id_list, vocab.size)
train_y = np.array(temp_train_toys.overall, dtype=np.int32)
train_y_b = temp_train_y
dev_x_csr = utils.id_lists_to_sparse_bow(x_tokens_id_list_dev, vocab.size)
dev_y = np.array(temp_dev_toys.overall, dtype=np.int32)
dev_y_b = temp_dev_y

print("Training set: x = {:s} sparse, y = {:s}".format(str(train_x_csr.shape), 
                                                str(train_y.shape)))
print("Test set:     x = {:s} sparse, y = {:s}".format(str(dev_x_csr.shape), 
                                                str(dev_y.shape)))

Training set: x = (1200000, 674074) sparse, y = (1200000,)
Test set:     x = (360000, 674074) sparse, y = (360000,)


In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

#Naive bayes for binary prediction
nb_b = MultinomialNB()
nb_b.fit(train_x_csr,train_y_b)
y_pred = nb_b.predict(dev_x_csr)

acc = accuracy_score(dev_y_b, y_pred)
print("Accuracy on dev set for binary prediction: {:.02%}".format(acc))

#Naive bayes for 4 level rating prediction, excluding the 3s
nb = MultinomialNB()
nb.fit(train_x_csr,train_y)
y_pred = nb.predict(dev_x_csr)

acc = accuracy_score(dev_y, y_pred)
print("Accuracy on dev set for 4 level (1,2,4,5) prediction: {:.02%}".format(acc))

Accuracy on dev set for binary prediction: 91.92%
Accuracy on dev set for 4 level (1,2,4,5) prediction: 71.24%


In [36]:

#Pulling out and printing most positive and negative features - binary prediction
linear_weights = nb_b.feature_log_prob_[1,] - nb_b.feature_log_prob_[0,]  # populate this with actual values
top_negative_features = np.argsort(linear_weights)[:20]
top_positive_features = np.argsort(linear_weights)[-20:]


print("Most negative features - binary prediction:")
for idx in top_negative_features:
    print("  {:s} ({:.02f})".format(vocab.id_to_word[idx], 
                                    linear_weights[idx]))
print("")
print("Most positive features: binary prediction:")
for idx in top_positive_features:
    print("  {:s} ({:.02f})".format(vocab.id_to_word[idx], 
                                    linear_weights[idx]))
    
#Pulling out and printing most positive and negative features - 4 way prediction
linear_weights = nb.feature_log_prob_[1,] - nb.feature_log_prob_[0,]  # populate this with actual values
top_negative_features = np.argsort(linear_weights)[:10]
top_positive_features = np.argsort(linear_weights)[-10:]


print("")
print("Most negative features - (1,2,4,5) prediction:")
for idx in top_negative_features:
    print("  {:s} ({:.02f})".format(vocab.id_to_word[idx], 
                                    linear_weights[idx]))
print("")
print("Most positive features: (1,2,4,5) prediction:")
for idx in top_positive_features:
    print("  {:s} ({:.02f})".format(vocab.id_to_word[idx], 
                                    linear_weights[idx]))

Most negative features - binary prediction:
  jpl-discount (-4.70)
  disgraceful (-4.57)
  .poor (-4.41)
  gbcpro (-4.16)
  99cent (-4.16)
  poor.the (-4.16)
  reimbursement (-4.09)
  disatisfied (-4.09)
  riddance (-4.09)
  bambin (-4.09)
  browsers (-4.09)
  v.motion (-4.01)
  returing (-4.01)
  3mp (-4.01)
  surge= (-4.01)
  gyp (-3.92)
  merchsource (-3.92)
  .total (-3.92)
  slaughterhouse (-3.92)
  aviva (-3.92)

Most positive features: binary prediction:
  gatherings (3.09)
  sweetest (3.11)
  ball-jointed (3.14)
  manatee (3.15)
  must-have (3.16)
  nitpick (3.19)
  samus (3.24)
  it.great (3.25)
  pullip (3.31)
  quality,10 (3.31)
  exia (3.31)
  please.the (3.33)
  meeple (3.44)
  non-gamers (3.44)
  ella (3.53)
  liger (3.55)
  seaside (3.65)
  excelente (3.69)
  gn (3.86)
  ball-hinged (4.00)

Most negative features - (1,2,4,5) prediction:
  carnivorous (-3.13)
  booker (-3.04)
  fraudulent (-2.98)
  nst (-2.94)
  v.motion (-2.79)
  jpl-discount (-2.79)
  flexpak (-2.65)
  

### Keeping track of results from test runs
With number in train set = 10000 (excl 3 ratings)    
    Accuracy on dev set for binary prediction: 88.74%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 67.16%
    Vocab Size : 38696
    
With number in train set = 50000 (excl 3 ratings)   
    Accuracy on dev set for binary prediction: 91.33%   
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 69.33% 
    Vocab Size : ~ 104000
    
With number in train set = 100000 (excl 3 ratings)
    Accuracy on dev set for binary prediction: 91.72%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 70.25%
    Vocab Size : 164897

With number in train set = 500000, dev set = 150000 (excl 3 ratings)    
    Accuracy on dev set for binary prediction: 91.92%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 71.24%
    vocab size 367096
    
With number in train set = 1200000, dev set = 360000 (excl 3 ratings)    
    Accuracy on dev set for binary prediction: 91.92%
    Accuracy on dev set for 4 level (1,2,4,5) prediction: 71.24%
    vocab size 674074